# Particle Tracking Tutorial using Trackpy

This notebook provides a comprehensive step-by-step workflow for particle tracking analysis using the trackpy library.

## Overview
- **Cell 0**: Setup and imports
- **Cell 1**: Load TIF file and display frame count
- **Cell 2**: Interactive frame viewer with intensity histogram
- **Cell 3**: Locate particles in a single frame
- **Cell 4**: Batch process multiple frames
- **Cell 5**: Link trajectories across frames

## Reference
This tutorial follows the trackpy walkthrough: https://soft-matter.github.io/trackpy/v0.7/tutorial/walkthrough.html

## Cell 0: Setup & Imports

Import all required libraries and configure display settings.

In [ ]:
# Core particle tracking and data handling
import trackpy as tp
import pims
import numpy as np
import pandas as pd

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Interactive widgets
from ipywidgets import interact, IntSlider, FloatSlider, FileUpload, Button, Output, VBox, HBox, Label
from IPython.display import display, HTML, clear_output

# File dialog for TIF file selection
import tkinter as tk
from tkinter import filedialog

# Warnings and display configuration
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ All libraries imported successfully")
print(f"Trackpy version: {tp.__version__}")

## Cell 1: Load TIF File & Display Frame Count

This cell provides a file browser to select a TIF file, loads it using pims for memory-efficient lazy loading, and displays the total number of frames.

In [ ]:
# Global variable to store frames
frames = None
tif_filepath = None

def browse_file():
    """Open file dialog to select TIF file"""
    global frames, tif_filepath
    
    # Create a root window and hide it
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    
    # Open file dialog
    filepath = filedialog.askopenfilename(
        title='Select TIF file',
        filetypes=[('TIF files', '*.tif *.tiff'), ('All files', '*.*')]
    )
    
    if filepath:
        try:
            # Load TIF file using pims (lazy loading)
            frames = pims.open(filepath)
            tif_filepath = filepath
            
            print(f"✓ File loaded successfully: {filepath}")
            print(f"  Total frames: {len(frames)}")
            print(f"  Frame shape: {frames[0].shape}")
            print(f"  Frame dtype: {frames[0].dtype}")
            
        except Exception as e:
            print(f"✗ Error loading file: {e}")
    else:
        print("No file selected")
    
    root.destroy()

# Alternative: FileUpload widget for Jupyter environments without tkinter
def load_from_upload(change):
    """Load TIF file from FileUpload widget"""
    global frames, tif_filepath
    
    if upload.value:
        # Get the uploaded file
        filename = list(upload.value.keys())[0]
        content = upload.value[filename]['content']
        
        # Save to temporary file
        import tempfile
        with tempfile.NamedTemporaryFile(delete=False, suffix='.tif') as tmp:
            tmp.write(content)
            tif_filepath = tmp.name
        
        try:
            # Load TIF file using pims
            frames = pims.open(tif_filepath)
            
            print(f"✓ File uploaded successfully: {filename}")
            print(f"  Total frames: {len(frames)}")
            print(f"  Frame shape: {frames[0].shape}")
            print(f"  Frame dtype: {frames[0].dtype}")
            
        except Exception as e:
            print(f"✗ Error loading file: {e}")

# Create UI
print("Choose your file loading method:")
print("\nOption 1: File Browser (recommended for local files)")
browse_button = Button(description='Browse Files')
browse_button.on_click(lambda x: browse_file())
display(browse_button)

print("\nOption 2: File Upload (for JupyterLab/cloud environments)")
upload = FileUpload(accept='.tif,.tiff', multiple=False)
upload.observe(load_from_upload, names='value')
display(upload)

## Cell 2: Interactive Frame Viewer with Intensity Histogram

This cell creates an interactive viewer to explore individual frames with:
- Frame selector to navigate through the TIF stack
- Interactive image display with zoom/pan using Plotly
- Intensity histogram showing the distribution of pixel values
- Min/max threshold sliders for image thresholding

The threshold values are stored for later use in particle detection.

In [ ]:
# Global variables for threshold values
min_threshold = 0
max_threshold = 255
selected_frame_idx = 0

def update_frame_viewer(frame_idx, min_thresh, max_thresh):
    """Display selected frame with histogram and threshold controls"""
    global min_threshold, max_threshold, selected_frame_idx
    
    if frames is None:
        print("Please load a TIF file first (Cell 1)")
        return
    
    # Store values globally
    min_threshold = min_thresh
    max_threshold = max_thresh
    selected_frame_idx = frame_idx
    
    # Get the selected frame
    frame = frames[frame_idx]
    
    # Apply thresholding
    frame_thresh = np.clip(frame, min_thresh, max_thresh)
    
    # Create subplot with frame and histogram side-by-side
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f'Frame {frame_idx} (Thresholded)', 'Intensity Histogram'),
        column_widths=[0.6, 0.4],
        horizontal_spacing=0.1
    )
    
    # Add frame image
    fig.add_trace(
        go.Heatmap(
            z=frame_thresh,
            colorscale='gray',
            showscale=True,
            colorbar=dict(x=0.58, len=0.8)
        ),
        row=1, col=1
    )
    
    # Add histogram
    hist_values, hist_bins = np.histogram(frame.flatten(), bins=100)
    fig.add_trace(
        go.Bar(
            x=hist_bins[:-1],
            y=hist_values,
            marker_color='steelblue',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Add threshold lines to histogram
    fig.add_vline(
        x=min_thresh, line_dash="dash", line_color="red",
        annotation_text=f"Min: {min_thresh}",
        row=1, col=2
    )
    fig.add_vline(
        x=max_thresh, line_dash="dash", line_color="green",
        annotation_text=f"Max: {max_thresh}",
        row=1, col=2
    )
    
    # Update layout
    fig.update_xaxes(title_text="X (pixels)", row=1, col=1)
    fig.update_yaxes(title_text="Y (pixels)", row=1, col=1, autorange='reversed')
    fig.update_xaxes(title_text="Intensity", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    
    fig.update_layout(
        height=500,
        width=1200,
        title_text=f"Frame Viewer - File: {tif_filepath if tif_filepath else 'N/A'}"
    )
    
    fig.show()
    
    # Print statistics
    print(f"Frame {frame_idx} statistics:")
    print(f"  Min intensity: {frame.min()}")
    print(f"  Max intensity: {frame.max()}")
    print(f"  Mean intensity: {frame.mean():.2f}")
    print(f"  Std intensity: {frame.std():.2f}")
    print(f"\nThreshold range: [{min_thresh}, {max_thresh}]")

if frames is not None:
    # Get intensity range from first frame
    first_frame = frames[0]
    min_val = int(first_frame.min())
    max_val = int(first_frame.max())
    
    # Create interactive viewer
    interact(
        update_frame_viewer,
        frame_idx=IntSlider(
            value=0,
            min=0,
            max=len(frames)-1,
            step=1,
            description='Frame:',
            continuous_update=False
        ),
        min_thresh=FloatSlider(
            value=min_val,
            min=min_val,
            max=max_val,
            step=(max_val-min_val)/100,
            description='Min Thresh:',
            continuous_update=False
        ),
        max_thresh=FloatSlider(
            value=max_val,
            min=min_val,
            max=max_val,
            step=(max_val-min_val)/100,
            description='Max Thresh:',
            continuous_update=False
        )
    )
else:
    print("Please load a TIF file first (Cell 1)")

## Cell 3: Locate Particles in Single Frame

This cell uses trackpy's `tp.locate()` function to detect particles in a single frame.

### Key Parameters:
- **diameter**: Particle size in pixels (must be odd integer)
- **minmass**: Minimum integrated brightness (filters out noise)
- **separation**: Minimum separation between particles (optional)
- **percentile**: Brightness percentile for background subtraction (optional)

### Outputs:
1. Annotated image showing detected particles
2. Mass distribution histogram (helps tune minmass)
3. Subpixel bias diagnostic (should be flat for good detection)

In [ ]:
# Global variable to store located particles
located_particles = None

def locate_particles_interactive(diameter, minmass, separation=None, percentile=64):
    """Locate particles in the selected frame with interactive parameters"""
    global located_particles
    
    if frames is None:
        print("Please load a TIF file first (Cell 1)")
        return
    
    # Ensure diameter is odd
    if diameter % 2 == 0:
        diameter += 1
        print(f"⚠ Diameter must be odd, using {diameter}")
    
    # Get the frame
    frame = frames[selected_frame_idx]
    
    # Set separation to diameter if not provided
    if separation is None:
        separation = diameter + 1
    
    print(f"Locating particles in frame {selected_frame_idx}...")
    print(f"Parameters: diameter={diameter}, minmass={minmass}, separation={separation}, percentile={percentile}")
    
    # Locate particles
    located_particles = tp.locate(
        frame,
        diameter=diameter,
        minmass=minmass,
        separation=separation,
        percentile=percentile
    )
    
    print(f"\n✓ Found {len(located_particles)} particles")
    
    # Create figure with 3 subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'Detected Particles (n={len(located_particles)})',
            'Mass Distribution',
            'Subpixel Bias Diagnostic',
            ''
        ),
        specs=[[{'type': 'heatmap'}, {'type': 'bar'}],
               [{'colspan': 2, 'type': 'scatter'}, None]],
        row_heights=[0.5, 0.5],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    # 1. Annotated image
    fig.add_trace(
        go.Heatmap(
            z=frame,
            colorscale='gray',
            showscale=True,
            colorbar=dict(x=0.47, len=0.4)
        ),
        row=1, col=1
    )
    
    # Add particle positions as scatter points
    if len(located_particles) > 0:
        fig.add_trace(
            go.Scatter(
                x=located_particles['x'],
                y=located_particles['y'],
                mode='markers',
                marker=dict(
                    size=diameter,
                    color='red',
                    symbol='circle-open',
                    line=dict(width=2)
                ),
                showlegend=False,
                hovertemplate='x: %{x:.1f}<br>y: %{y:.1f}<extra></extra>'
            ),
            row=1, col=1
        )
    
    # 2. Mass distribution histogram
    if len(located_particles) > 0:
        fig.add_trace(
            go.Histogram(
                x=located_particles['mass'],
                nbinsx=50,
                marker_color='steelblue',
                showlegend=False
            ),
            row=1, col=2
        )
        
        # Add minmass threshold line
        fig.add_vline(
            x=minmass,
            line_dash="dash",
            line_color="red",
            annotation_text=f"minmass={minmass}",
            row=1, col=2
        )
    
    # 3. Subpixel bias diagnostic
    if len(located_particles) > 0:
        # Calculate subpixel bias
        x_bias = located_particles['x'] % 1
        y_bias = located_particles['y'] % 1
        
        fig.add_trace(
            go.Histogram(
                x=x_bias,
                nbinsx=20,
                name='X bias',
                marker_color='blue',
                opacity=0.7
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Histogram(
                x=y_bias,
                nbinsx=20,
                name='Y bias',
                marker_color='red',
                opacity=0.7
            ),
            row=2, col=1
        )
    
    # Update axes
    fig.update_xaxes(title_text="X (pixels)", row=1, col=1)
    fig.update_yaxes(title_text="Y (pixels)", autorange='reversed', row=1, col=1)
    fig.update_xaxes(title_text="Mass", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    fig.update_xaxes(title_text="Subpixel Position", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    
    fig.update_layout(
        height=900,
        width=1200,
        title_text=f"Particle Detection Results - Frame {selected_frame_idx}"
    )
    
    fig.show()
    
    # Display particle statistics
    if len(located_particles) > 0:
        print("\nParticle Statistics:")
        print(f"  Mass range: [{located_particles['mass'].min():.1f}, {located_particles['mass'].max():.1f}]")
        print(f"  Mean mass: {located_particles['mass'].mean():.1f}")
        print(f"  Size range: [{located_particles['size'].min():.2f}, {located_particles['size'].max():.2f}]")
        print(f"  Mean size: {located_particles['size'].mean():.2f}")
        
        # Check subpixel bias flatness
        x_bias_std = (located_particles['x'] % 1).std()
        y_bias_std = (located_particles['y'] % 1).std()
        print(f"\nSubpixel bias check:")
        print(f"  X bias std: {x_bias_std:.3f} (should be ~0.29 for uniform distribution)")
        print(f"  Y bias std: {y_bias_std:.3f} (should be ~0.29 for uniform distribution)")
        
        if x_bias_std < 0.2 or y_bias_std < 0.2:
            print("  ⚠ Warning: Subpixel bias is not flat - consider adjusting diameter")
        else:
            print("  ✓ Subpixel bias looks good")

if frames is not None:
    # Create interactive controls
    interact(
        locate_particles_interactive,
        diameter=IntSlider(
            value=11,
            min=3,
            max=51,
            step=2,
            description='Diameter:',
            continuous_update=False
        ),
        minmass=FloatSlider(
            value=100,
            min=0,
            max=10000,
            step=50,
            description='Min Mass:',
            continuous_update=False
        ),
        separation=IntSlider(
            value=None,
            min=1,
            max=100,
            step=1,
            description='Separation:',
            continuous_update=False
        ),
        percentile=IntSlider(
            value=64,
            min=0,
            max=100,
            step=1,
            description='Percentile:',
            continuous_update=False
        )
    )
else:
    print("Please load a TIF file first (Cell 1)")

## Cell 4: Batch Processing Multiple Frames

This cell uses `tp.batch()` to locate particles across multiple frames efficiently.

### Features:
- Process a range of frames or all frames in the TIF stack
- Display progress indicator during processing
- Show summary statistics and visualizations
- Store results for trajectory linking

### Performance Note:
For large datasets, consider processing a subset of frames first to tune parameters.

In [ ]:
# Global variable to store batch results
batch_results = None

def batch_process_frames(start_frame, end_frame, diameter, minmass, separation=None, percentile=64):
    """Batch process multiple frames to locate particles"""
    global batch_results
    
    if frames is None:
        print("Please load a TIF file first (Cell 1)")
        return
    
    # Ensure diameter is odd
    if diameter % 2 == 0:
        diameter += 1
        print(f"⚠ Diameter must be odd, using {diameter}")
    
    # Validate frame range
    start_frame = max(0, start_frame)
    end_frame = min(len(frames) - 1, end_frame)
    
    if separation is None:
        separation = diameter + 1
    
    print(f"Batch processing frames {start_frame} to {end_frame}...")
    print(f"Parameters: diameter={diameter}, minmass={minmass}, separation={separation}, percentile={percentile}")
    print(f"Total frames to process: {end_frame - start_frame + 1}")
    print("\nProcessing...")
    
    # Batch locate particles with progress output
    batch_results = tp.batch(
        frames[start_frame:end_frame+1],
        diameter=diameter,
        minmass=minmass,
        separation=separation,
        percentile=percentile,
        processes=1  # Use single process to avoid issues in notebooks
    )
    
    # Adjust frame numbers to match original indexing
    batch_results['frame'] = batch_results['frame'] + start_frame
    
    print(f"\n✓ Batch processing complete!")
    print(f"  Total particles found: {len(batch_results)}")
    print(f"  Frames processed: {batch_results['frame'].nunique()}")
    
    # Calculate particles per frame
    particles_per_frame = batch_results.groupby('frame').size()
    
    print(f"\nParticles per frame statistics:")
    print(f"  Mean: {particles_per_frame.mean():.1f}")
    print(f"  Std: {particles_per_frame.std():.1f}")
    print(f"  Min: {particles_per_frame.min()}")
    print(f"  Max: {particles_per_frame.max()}")
    
    # Create visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Particles per Frame', 'Mass Distribution (All Frames)'),
        horizontal_spacing=0.15
    )
    
    # Particles per frame plot
    fig.add_trace(
        go.Scatter(
            x=particles_per_frame.index,
            y=particles_per_frame.values,
            mode='lines+markers',
            marker=dict(size=6, color='steelblue'),
            line=dict(color='steelblue'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Add mean line
    fig.add_hline(
        y=particles_per_frame.mean(),
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: {particles_per_frame.mean():.1f}",
        row=1, col=1
    )
    
    # Mass distribution
    fig.add_trace(
        go.Histogram(
            x=batch_results['mass'],
            nbinsx=50,
            marker_color='steelblue',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Update axes
    fig.update_xaxes(title_text="Frame Number", row=1, col=1)
    fig.update_yaxes(title_text="Particle Count", row=1, col=1)
    fig.update_xaxes(title_text="Mass", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    
    fig.update_layout(
        height=500,
        width=1200,
        title_text=f"Batch Processing Results (Frames {start_frame}-{end_frame})"
    )
    
    fig.show()
    
    # Display sample of results
    print("\nSample of detected particles (first 10 rows):")
    display(batch_results.head(10))

if frames is not None:
    # Create interactive controls
    interact(
        batch_process_frames,
        start_frame=IntSlider(
            value=0,
            min=0,
            max=len(frames)-1,
            step=1,
            description='Start Frame:',
            continuous_update=False
        ),
        end_frame=IntSlider(
            value=min(len(frames)-1, 49),  # Default to first 50 frames
            min=0,
            max=len(frames)-1,
            step=1,
            description='End Frame:',
            continuous_update=False
        ),
        diameter=IntSlider(
            value=11,
            min=3,
            max=51,
            step=2,
            description='Diameter:',
            continuous_update=False
        ),
        minmass=FloatSlider(
            value=100,
            min=0,
            max=10000,
            step=50,
            description='Min Mass:',
            continuous_update=False
        ),
        separation=IntSlider(
            value=None,
            min=1,
            max=100,
            step=1,
            description='Separation:',
            continuous_update=False
        ),
        percentile=IntSlider(
            value=64,
            min=0,
            max=100,
            step=1,
            description='Percentile:',
            continuous_update=False
        )
    )
else:
    print("Please load a TIF file first (Cell 1)")

## Cell 5: Link Trajectories

This cell uses `tp.link()` to connect particles across frames into trajectories.

### Key Parameters:
- **search_range**: Maximum distance a particle can move between frames (in pixels)
- **memory**: Number of frames a particle can disappear and still be linked to the same trajectory

### Features:
- Link particles into trajectories
- Display trajectory statistics
- Visualize trajectories interactively with Plotly
- Filter spurious short tracks
- Export trajectory data to CSV

In [ ]:
# Global variable to store linked trajectories
trajectories = None
filtered_trajectories = None

def link_trajectories(search_range, memory, min_track_length=5):
    """Link particles across frames into trajectories"""
    global trajectories, filtered_trajectories
    
    if batch_results is None:
        print("Please run batch processing first (Cell 4)")
        return
    
    print(f"Linking trajectories...")
    print(f"Parameters: search_range={search_range}, memory={memory}")
    print(f"Input: {len(batch_results)} particles across {batch_results['frame'].nunique()} frames")
    
    # Link trajectories
    trajectories = tp.link(batch_results, search_range=search_range, memory=memory)
    
    print(f"\n✓ Linking complete!")
    print(f"  Total trajectories: {trajectories['particle'].nunique()}")
    
    # Filter short trajectories
    print(f"\nFiltering trajectories shorter than {min_track_length} frames...")
    filtered_trajectories = tp.filter_stubs(trajectories, threshold=min_track_length)
    
    # Calculate trajectory statistics
    track_lengths = filtered_trajectories.groupby('particle').size()
    
    print(f"\nTrajectory statistics (after filtering):")
    print(f"  Number of trajectories: {len(track_lengths)}")
    print(f"  Mean track length: {track_lengths.mean():.1f} frames")
    print(f"  Median track length: {track_lengths.median():.1f} frames")
    print(f"  Min track length: {track_lengths.min()} frames")
    print(f"  Max track length: {track_lengths.max()} frames")
    print(f"  Removed {trajectories['particle'].nunique() - len(track_lengths)} short trajectories")
    
    # Create visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Track Length Distribution',
            'Trajectory Overlay (Sample)',
            'Particle Displacement',
            'Trajectory Start Positions'
        ),
        specs=[
            [{'type': 'bar'}, {'type': 'scatter'}],
            [{'type': 'scatter'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.12
    )
    
    # 1. Track length histogram
    fig.add_trace(
        go.Histogram(
            x=track_lengths.values,
            nbinsx=50,
            marker_color='steelblue',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Trajectory overlay (sample of trajectories for clarity)
    sample_particles = track_lengths.nlargest(min(20, len(track_lengths))).index
    
    for particle_id in sample_particles:
        traj = filtered_trajectories[filtered_trajectories['particle'] == particle_id]
        fig.add_trace(
            go.Scatter(
                x=traj['x'],
                y=traj['y'],
                mode='lines+markers',
                marker=dict(size=4),
                line=dict(width=1),
                showlegend=False,
                hovertemplate=f'Particle {particle_id}<br>x: %{{x:.1f}}<br>y: %{{y:.1f}}<extra></extra>'
            ),
            row=1, col=2
        )
    
    # 3. Displacement plot
    # Calculate frame-to-frame displacement for each trajectory
    for particle_id in sample_particles:
        traj = filtered_trajectories[filtered_trajectories['particle'] == particle_id].sort_values('frame')
        if len(traj) > 1:
            dx = traj['x'].diff()
            dy = traj['y'].diff()
            displacement = np.sqrt(dx**2 + dy**2)
            
            fig.add_trace(
                go.Scatter(
                    x=traj['frame'].iloc[1:],
                    y=displacement.iloc[1:],
                    mode='lines',
                    line=dict(width=1),
                    showlegend=False,
                    hovertemplate=f'Particle {particle_id}<br>Frame: %{{x}}<br>Displacement: %{{y:.2f}}<extra></extra>'
                ),
                row=2, col=1
            )
    
    # 4. Start positions
    start_positions = filtered_trajectories.groupby('particle').first()
    fig.add_trace(
        go.Scatter(
            x=start_positions['x'],
            y=start_positions['y'],
            mode='markers',
            marker=dict(
                size=6,
                color=track_lengths.values,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title='Track Length', x=1.15)
            ),
            showlegend=False,
            hovertemplate='Particle: %{text}<br>x: %{x:.1f}<br>y: %{y:.1f}<extra></extra>',
            text=start_positions.index
        ),
        row=2, col=2
    )
    
    # Update axes
    fig.update_xaxes(title_text="Track Length (frames)", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_xaxes(title_text="X (pixels)", row=1, col=2)
    fig.update_yaxes(title_text="Y (pixels)", autorange='reversed', row=1, col=2)
    fig.update_xaxes(title_text="Frame", row=2, col=1)
    fig.update_yaxes(title_text="Displacement (pixels)", row=2, col=1)
    fig.update_xaxes(title_text="X (pixels)", row=2, col=2)
    fig.update_yaxes(title_text="Y (pixels)", autorange='reversed', row=2, col=2)
    
    fig.update_layout(
        height=900,
        width=1300,
        title_text=f"Trajectory Analysis (Showing {len(sample_particles)} longest trajectories)"
    )
    
    fig.show()
    
    # Display sample trajectories
    print("\nSample of trajectory data (first 10 rows):")
    display(filtered_trajectories.head(10))

def export_trajectories(filename='trajectories.csv'):
    """Export trajectory data to CSV"""
    if filtered_trajectories is None:
        print("No trajectories to export. Please run trajectory linking first.")
        return
    
    filtered_trajectories.to_csv(filename, index=False)
    print(f"✓ Trajectories exported to {filename}")
    print(f"  Total rows: {len(filtered_trajectories)}")
    print(f"  Columns: {', '.join(filtered_trajectories.columns)}")

if batch_results is not None:
    # Create interactive controls for linking
    interact(
        link_trajectories,
        search_range=FloatSlider(
            value=5.0,
            min=1.0,
            max=50.0,
            step=0.5,
            description='Search Range:',
            continuous_update=False
        ),
        memory=IntSlider(
            value=3,
            min=0,
            max=20,
            step=1,
            description='Memory:',
            continuous_update=False
        ),
        min_track_length=IntSlider(
            value=5,
            min=1,
            max=50,
            step=1,
            description='Min Length:',
            continuous_update=False
        )
    )
    
    # Export button
    print("\n" + "="*50)
    print("Export Trajectories:")
    export_button = Button(description='Export to CSV')
    export_button.on_click(lambda x: export_trajectories('trajectories.csv'))
    display(export_button)
else:
    print("Please run batch processing first (Cell 4)")

## Additional Analysis (Optional)

After completing the basic workflow, you can perform additional analyses:

### Mean Squared Displacement (MSD)
```python
# Calculate ensemble MSD
em = tp.emsd(filtered_trajectories, mpp=0.16, fps=30)  # Adjust mpp and fps for your data

# Plot MSD
fig = go.Figure()
fig.add_trace(go.Scatter(x=em.index, y=em['msd'], mode='lines+markers'))
fig.update_layout(title='Ensemble Mean Squared Displacement',
                  xaxis_title='Lag time (frames)',
                  yaxis_title='MSD (μm²)')
fig.show()
```

### Velocity Distribution
```python
# Calculate velocities
velocities = []
for particle_id in filtered_trajectories['particle'].unique():
    traj = filtered_trajectories[filtered_trajectories['particle'] == particle_id].sort_values('frame')
    if len(traj) > 1:
        dx = traj['x'].diff()
        dy = traj['y'].diff()
        v = np.sqrt(dx**2 + dy**2)
        velocities.extend(v.dropna().values)

# Plot velocity distribution
fig = go.Figure()
fig.add_trace(go.Histogram(x=velocities, nbinsx=50))
fig.update_layout(title='Velocity Distribution',
                  xaxis_title='Velocity (pixels/frame)',
                  yaxis_title='Count')
fig.show()
```

### Individual Trajectory Plots
```python
# Plot specific trajectories
particle_id = 0  # Change to desired particle ID
traj = filtered_trajectories[filtered_trajectories['particle'] == particle_id]

fig = go.Figure()
fig.add_trace(go.Scatter(x=traj['x'], y=traj['y'], mode='lines+markers',
                        marker=dict(color=traj['frame'], colorscale='Viridis', showscale=True)))
fig.update_layout(title=f'Trajectory of Particle {particle_id}',
                  xaxis_title='X (pixels)',
                  yaxis_title='Y (pixels)')
fig.update_yaxes(autorange='reversed')
fig.show()
```